In [71]:
import csv
import pandas as pd
from sdv.lite import TabularPreset
from sdv.evaluation import evaluate



In [72]:
config = pd.read_json("config.json") #config file(dictionary)
data_file = config["Input File"][0]
print(data_file)
target_file = config["Output File"][0]
data = pd.read_csv(str(data_file))
headers = data.columns

feature_dict = config["Features"]

features = [] #names of relevant headers
for feature in config["Features"].keys():
    features.append(feature) # wanted headers

allowed_percentage = config["Percentage"][0]
number_of_samples = int(config["N_samples"][0])
#print(features)


combined.csv


In [73]:

#Remove unspecified columns
for header in headers:
    if not header in features:
        data.pop(header)
        #print("Deleting column", header, "because of configuration.")

#Remove columns with to many NaNs
headers = data.columns #fixed list of columns we need
for header in headers:
    nan_percentage = data[header].isna().sum()/len(data[header].index)
    if nan_percentage > allowed_percentage:
        data.pop(header)
        headers = headers.drop(header)
        print("Deleting column", header, "because of" ,"{:.2%}".format(nan_percentage), "NaNs.")

#removing unwanted nans and interpolating others

numericals = []
for x in headers:
    if feature_dict[x][0] == "numerical":
        numericals.append(x)

for numerical in numericals:
    data[numerical] = pd.to_numeric(data[numerical], errors='coerce')

data.interpolate(method='pad')






,Cancer Type Detailed,Mutation Count,Oncotree Code,Overall Survival (Months),Sample Type,Diagnosis Age,Neoplasm Histologic Grade,Treatment Details before PDX,TERT,IDH1,PABPC3,PTEN
0,Glioblastoma Multiforme,173.0,GBM,30.00,Recurrence,40.0,II,RT,Thalidomide,Carboplatin,WT,WT
1,Glioblastoma Multiforme,1756.0,GBM,21.00,Recurrence,67.0,IV,RT,WT,WT,E2101K,P191T I195V
2,Glioblastoma Multiforme,158.0,GBM,8.04,Primary,62.0,IV,None,WT,WT,K312R T319I,I101T
3,Glioblastoma Multiforme,161.0,GBM,25.92,Primary,64.0,IV,None,WT,WT,V293G A313V K312R A154G K156R,WT
4,Glioblastoma Multiforme,669.0,GBM,44.88,Recurrence,65.0,IV,RT,WT,WT,WT,A313Rfs*14 F335Lfs*19 M326Gfs*21 K231E P191T E...
...,...,...,...,...,...,...,...,...,...,...,...,...
1537,Oligodendroglioma,8.0,ODG,164.00,Recurrence,29.0,G2,Treatment Details before PDX,WT,WT,NS,WT
1538,Anaplastic Oligodendroglioma,4.0,AODG,88.10,Primary,36.0,G2,Treatment Details before PDX,Promoter,R132H,NS,WT
1539,Glioblastoma Multiforme,12.0,GBM,41.20,Primary,55.0,G2,Treatment Details before PDX,Promoter,WT,NS,S170N
1540,Anaplastic Astrocytoma,11.0,AASTR,14.00,Recurrence,26.0,G2,Treatment Details before PDX,Promoter,R132C,NS,WT


In [74]:
#create validation set

validation_set = data.sample(frac=0.3)
#removal = data.sample(frac=0.9)

#data = data.drop(removal.index)

validation_set.reset_index()

print(len(data))


1542


In [75]:
#Generating Metadata

metadata = {}
metadata["fields"] = {}

for x in headers:
    metadata["fields"][x] = {}
    metadata["fields"][x]["type"] = feature_dict[x][0]
    if feature_dict[x][0] == "numerical":
        metadata["fields"][x]["subtype"] = feature_dict[x][1]
metadata["constraints"] = []

#print(metadata)



In [76]:
#fit a model to the data and generating data
#model = TabularPreset(name='FAST_ML', metadata=metadata)
def makeData():
    model = TabularPreset(name='FAST_ML', metadata=metadata)

    model.fit(data)

    new_data = model.sample(num_rows=number_of_samples)

    new_data = new_data.round(decimals = 2)

    new_data.to_csv(target_file, index=False)

    return "{:.2%}".format(evaluate(new_data, validation_set, metrics=['KSTest']))

    #print("The generated data is", "{:.2%}".format(evaluate(new_data, validation_set, metrics=['KSTest'])), "accurate to the original data. If this is unsatisfactory try using less or different features, more data or removing more NaNs.")

In [77]:
from dataclasses import dataclass


dataframe = []

for x in range(0,1000):
    dataframe.append(makeData())


file = open("100%NaNs_Interpolated_18Features_1542InputLines.csv", "w", newline="")

writer = csv.writer(file)

for val in dataframe:
    writer.writerow([val])


file.close()

In [78]:
import session_info

session_info.show()